### View spectrogram (whitened or regular)

In [ ]:
import numpy as np
import subjects

sessions = subjects.sd.ratUday4

In [ ]:
from neuropy.utils.signal_process import FourierSg
from hfuncs import whiten_signal

for sub,sess in enumerate(sessions):
    signal = sess.eegfile.get_signal(82)
    signal = whiten_signal(signal)
    spect = FourierSg(signal,window=60,overlap=10)

In [ ]:
%matplotlib widget
from neuropy.plotting import plot_signal_heatmap
import matplotlib.pyplot as plt

_, ax = plt.subplots()
ax.imshow(
    spect.traces,
    aspect="auto",
    vmax=0.004,
    cmap="jet",
    origin="lower",
    extent=[spect.time[0], spect.time[-1], spect.freqs[0], spect.freqs[-1]],
)


In [ ]:
%matplotlib widget
from neuropy.plotting import plot_signal_heatmap
import matplotlib.pyplot as plt

_, ax = plt.subplots()
ax.imshow(
    spect.traces,
    aspect="auto",
    vmax=0.004,
    cmap="jet",
    origin="lower",
    extent=[spect.time[0], spect.time[-1], spect.freqs[0], spect.freqs[-1]],
)


### Averaged slow oscillation power during SD-POST

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
from scipy import stats
import subjects
import pandas as pd

sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

In [ ]:
spects = []
for sess in sessions:
    t_start, t_stop = sess.paradigm["post"].flatten()
    signal = sess.eegfile.get_signal(113, t_start=t_start, t_stop=t_start+15*60)
    spects.append(signal_process.FourierSg(signal, window=5 * 60, overlap=60))


In [ ]:
fig = plotting.Fig(grid=(4, 3), hspace=0.3, wspace=0.35)
# fig.suptitle("Figure 1", x=0, ha="left", fontsize=16)

# ------ panel a --------
gs_spect = fig.subplot2grid(fig.gs[0, :2], grid=(2, 1))
brainstates = [
    sess.brainstates.time_slice(
        sess.paradigm["post"].flatten()[0], sess.paradigm["post"].flatten()[1]
    )
    for sess in [sd_sessions[1], nsd_sessions[1]]
]
t_starts = [
    sess.paradigm["post"].flatten()[0] for sess in [sd_sessions[1], nsd_sessions[1]]
]

for i, spect in enumerate(spects):
    gs_swa = fig.subplot2grid(gs_spect[i], grid=(4, 1))
    ax_theta = plt.subplot(gs_swa[1:])
    ax_hypno = plt.subplot(gs_swa[0])
    plotting.plot_hypnogram(brainstates[i], ax=ax_hypno)

    if i == 0:
        ax_theta.set_xlabel("")
        ax_theta.set_xticklabels([])

### Best slow wave channels based on bimodality of slope

In [ ]:
import numpy as np
import subjects

sessions = subjects.sd.ratSday3

In [ ]:
from neuropy.utils.signal_process import FourierSg
from hfuncs import whiten_signal
import scipy.signal as sg
from numpy.polynomial import polynomial
from scipy.ndimage import gaussian_filter1d

for sub, sess in enumerate(sessions):

    # channels = [19,25,2,1,93,87,189,183]
    channels = np.arange(100)
    slopes = []
    for chan in channels:
        signal = sess.eegfile.get_signal(chan)
        # spect = FourierSg(signal,window=60,overlap=10)
        f, t, spect = sg.spectrogram(
            signal.traces[0], fs=1250, nperseg=60 * 1250, noverlap=10 * 1250,axis=-1
        )
        spect = gaussian_filter1d(spect,sigma=4,axis=0)

        indices = (f>4) & (f<90)
        f = np.log10(f[indices])
        spect = np.log10(spect[indices,:])

        invalid_col = ~np.isfinite(spect).any(axis=0)
        linfit = np.polyfit(f,spect[:,~invalid_col],deg=1)
        slopes.append(linfit[0])


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.signal import find_peaks
# bins = np.linspace(1,3,100)
_,ax = plt.subplots(1,1)

metric=[]
for i,slp in enumerate(slopes):
    # hist_slopes = np.histogram(gaussian_filter1d(np.abs(slp),sigma=2),bins=bins)[0]
    kde = sm.nonparametric.KDEUnivariate(gaussian_filter1d(np.abs(slp),sigma=2))
    kde.fit()

    peaks_ind = find_peaks(kde.density)[0]

    if len(peaks_ind)>1:
        peaks_ind = peaks_ind[:2]
        peaks_loc = kde.support[peaks_ind]
        h1,h2 = kde.density[peaks_ind] 

        between_peak = kde.density[peaks_ind[0]:peaks_ind[1]]
        dip_ind = find_peaks(-1 * between_peak)[0]+ peaks_ind[0]
        dip_loc = kde.support[dip_ind]
        dip_height = kde.density[dip_ind]

        param1 =  np.abs(h1-h2)/(h1+h2)
        param2 = np.diff(peaks_loc)

        metric.append([param1,param2])
        ax.plot(kde.support,kde.density,label=channels[i])

    else:

        ax.plot(kde.support,kde.density,label=channels[i])
#     ax.axvline(peaks_loc[0])
#     ax.axvline(peaks_loc[1])
#     ax.axvline(dip_loc)

# ax.legend()

In [ ]:
%matplotlib widget
metric1 = np.asarray(metric)

plt.plot(metric1[:,0],metric1[:,1],'.')



In [ ]:
%matplotlib widget
from neuropy.plotting import plot_signal_heatmap
import matplotlib.pyplot as plt

_, ax = plt.subplots()
ax.imshow(
    spect,
    aspect="auto",
    vmax=400,
    cmap="jet",
    origin="lower",
    extent=[t[0], t[-1], f[0], f[-1]],
)

### Pooled detection hippocampal slow waves 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
from scipy import stats
import subjects
import pandas as pd

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

# bottom channels (radiatum channels)
channels = [23,103,183,111,79,23,103,183,191,127,55]
# channels = [91, 77, 32, 57, 20, 13, 45, 32, 57, 7, 83]

assert len(channels) == len(sessions)

In [ ]:
from tqdm.notebook import tqdm

from neuropy.analyses.oscillations import detect_hpc_slow_wave_epochs

for sub,sess in enumerate(tqdm(sessions)):

    signal = sess.eegfile.get_signal(channels[sub])
    states = sess.brainstates['nrem'].duration_slice(min_dur=60)
    epochs = detect_hpc_slow_wave_epochs(signal,(0.5,4),nrem_epochs=states)
    epochs.save(sess.filePrefix.with_suffix('.sw'))

### CCG between Interneurons and hpc slow waves

In [ ]:
import numpy as np
import subjects

sessions = subjects.sd.pf_sess
# channels = [23,103,183,191,127,55]
channels = [13, 45, 32, 57, 7, 83]


In [ ]:
from neuropy.utils.ccg import correlograms
from neuropy.core import Epoch

ccgs_all = []
avg_sw = []
for sub, sess in enumerate(sessions):
    sw = sess.sw.starts
    sw_id = np.ones(len(sw))
    neurons = sess.neurons.get_neuron_type("inter")

    
    trace = sess.eegfile.get_signal(channels[sub]).traces[0]
    frames = np.concatenate([np.arange(e-500,e+500) for e in (sw*1250).astype('int')])
    avg_sw_sub = np.mean(trace[frames].reshape(len(sw),-1),axis=0,keepdims=True)
    avg_sw.append(np.tile(avg_sw_sub,(neurons.n_neurons,1)))

    def _get_ccg(spikes):
        clus = np.concatenate([sw_id, 2 * np.ones(len(spikes))]).astype('int')
        trains = np.concatenate([sw, spikes])
        sort_ind = np.argsort(trains)
        ccgs = correlograms(
            trains[sort_ind],
            clus[sort_ind],
            bin_size=0.010,
            window_size=0.8,
            sample_rate=30000,
        )
        return ccgs[0,1,:]
    
    ccgs_all.append(np.asarray([_get_ccg(_) for _ in neurons.spiketrains]))

ccgs_all = np.vstack(ccgs_all)
avg_sw = np.vstack(avg_sw)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from scipy import stats

_,axs= plt.subplots(13,5,sharex=True)
axs= axs.reshape(-1)
t = np.linspace(-405,405,ccgs_all.shape[1]+1)
t2 = np.linspace(-400,400,avg_sw.shape[1])
for i,ax in enumerate(axs):

    ax.stairs(ccgs_all[i]/np.mean(ccgs_all[i]),t,fill=True)
    ax.plot(t2,avg_sw[i]/np.sum(avg_sw[i])+0.5,'r')